<a href="https://colab.research.google.com/github/pomeloblue/GPT-SoVITS/blob/main/%E5%9B%BE%E6%96%87%E6%A3%80%E7%B4%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import torch
from PIL import Image
from transformers import BertTokenizer, BertModel
from torchvision import transforms, models
from tqdm import tqdm
import time
from google.colab import files

# GPU加速
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 检查文件
print("Current directory contents:")
print(os.listdir())

Using device: cuda
Current directory contents:
['.config', 'sample_data']


1: 导入必要的库和设置

In [2]:
def check_and_upload_file(filename):
    if not os.path.exists(filename):
        print(f"{filename} not found. Please upload it.")
        uploaded = files.upload()
        if filename not in uploaded:
            print(f"Failed to upload {filename}. Please try again.")
            return False
    return True

# 检查并上传必要的文件
required_files = ['train.csv', 'test_query.csv']
for file in required_files:
    if not check_and_upload_file(file):
        raise Exception(f"Failed to upload {file}")

# 检查图像文件夹
image_folder = 'image'
if not os.path.exists(image_folder):
    print(f"'{image_folder}' folder not found. Please upload a zip file containing the image folder.")
    uploaded = files.upload()
    if 'image.zip' in uploaded:
        !unzip -q image.zip
        print("Image folder extracted.")
    else:
        raise Exception("Failed to upload image.zip")

# 加载数据
train_df = pd.read_csv('train.csv', sep='\t')
test_df = pd.read_csv('test_query.csv')
print("Train data sample:")
print(train_df.head())
print("\nTrain data columns:")
print(train_df.columns)
print("\nTest data sample:")
print(test_df.head())
print("\nTest data columns:")
print(test_df.columns)

NameError: name 'os' is not defined

2:检查文件是否存在和上传函数

In [ ]:
# 图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 文本预处理
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# 文本特征提取
bert_model = BertModel.from_pretrained('bert-base-chinese').to(device)

def extract_text_features(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :]

# 图像特征提取
resnet = models.resnet50(pretrained=True).to(device)
resnet.fc = torch.nn.Identity()

def extract_image_features(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            return resnet(img_tensor)
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None

class MultimodalModel(torch.nn.Module):
    def __init__(self, text_dim, img_dim, output_dim):
        super().__init__()
        self.text_projection = torch.nn.Linear(text_dim, output_dim)
        self.img_projection = torch.nn.Linear(img_dim, output_dim)

    def forward(self, text_features, img_features):
        text_output = self.text_projection(text_features) if text_features is not None else None
        img_output = self.img_projection(img_features) if img_features is not None else None
        return text_output, img_output

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]


 3:模型和特征提取函数

In [ ]:
def main():
    # 加载数据
    train_df = pd.read_csv('train.csv', sep='\t')
    test_df = pd.read_csv('test_query.csv')
    print("Train data sample:")
    print(train_df.head())
    print("\nTrain data columns:")
    print(train_df.columns)
    print("\nTest data sample:")
    print(test_df.head())
    print("\nTest data columns:")
    print(test_df.columns)

    model = MultimodalModel(768, 2048, 512).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = torch.nn.CosineSimilarity()

    num_epochs = 15
    total_rows = len(train_df)

    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1}/{num_epochs}")
        skipped_rows = 0
        processed_rows = 0
        epoch_loss = 0

        for index, row in tqdm(train_df.iterrows(), total=total_rows, desc=f"Epoch {epoch+1}"):
            title = str(row['title'])
            path = str(row['path'])

            text_features = extract_text_features(title)
            img_features = extract_image_features(os.path.join(image_folder, path))

            if img_features is None:
                skipped_rows += 1
                continue

            text_output, img_output = model(text_features, img_features)
            loss = 1 - criterion(text_output, img_output)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            processed_rows += 1

            if processed_rows % 1000 == 0:
                elapsed_time = time.time() - start_time
                print(f"Processed {processed_rows} rows in {elapsed_time:.2f} seconds")

        if processed_rows > 0:
            avg_loss = epoch_loss / processed_rows
            print(f"Epoch {epoch+1}/{num_epochs} completed")
            print(f"Average loss: {avg_loss:.4f}")
        else:
            print(f"Epoch {epoch+1}/{num_epochs} completed")
            print("No rows were processed in this epoch")
        print(f"Skipped rows in this epoch: {skipped_rows}")

    print("Training completed")
    print(f"Total time elapsed: {time.time() - start_time:.2f} seconds")

    return model, criterion, test_df

if __name__ == "__main__":
    trained_model, trained_criterion, test_df = main()

Train data sample:
                                title  \
0  云思木想2019秋装新款高领弹力前后打揽修身织带长袖T恤女36723   
1          闺蜜裤2019年春夏款减龄双杠棉质弹力打底裤外穿显瘦   
2  画瓷 2019夏季新款小清新chic一字肩上衣女超仙透明网纱蕾丝小衫   
3    帕波仕蒂海宁真皮衣男夹克水貂毛领绵羊皮羽绒服2019新款休闲外套   
4    实拍#大码女装胖mm春夏新款显瘦连衣裙 减龄赫本中长款复古小黑裙   

                                       path  
0  0ec90019-eea7-4285-adf2-a4a5bb85198b.jpg  
1  67257121-c77d-4529-9b21-23a04da0a8eb.jpg  
2  49a5f2bd-03a0-4a74-9c0a-e97849d31262.jpg  
3  307151b3-d488-4bc0-aea8-64c7630b291f.jpg  
4  e93fe9dd-a6fb-438c-b316-c5c70e7e59cc.jpg  

Train data columns:
Index(['title', 'path'], dtype='object')

Test data sample:
                                title
0    雪纺半身裙夏女2019新款韩版中长款裙子波点A字裙百褶裙薄款半裙
1     程茧儿 秋冬新款ZIM超仙流苏印花系带荷叶边性感拼接长袖连衣裙
2  福利款【格】2019夏新款毛边牛仔裙a字裙气质半身裙百搭短裙0393
3    妈妈装夏装套装小衫上衣服中老年女装阔太太大码两件套洋气40岁50
4  台装春夏新品 洪前气质印花背部经典条纹短袖夏季清凉T恤女86287Q

Test data columns:
Index(['title'], dtype='object')
Starting epoch 1/15


Epoch 1:  20%|██        | 1004/5000 [00:23<01:16, 52.07it/s]

Processed 1000 rows in 23.75 seconds


Epoch 1:  40%|████      | 2005/5000 [00:45<01:01, 48.41it/s]

Processed 2000 rows in 45.54 seconds


...

Processed 3000 rows in 1620.17 seconds


Epoch 15:  80%|████████  | 4005/5000 [01:27<00:25, 39.16it/s]

Processed 4000 rows in 1641.46 seconds


Epoch 15: 100%|██████████| 5000/5000 [01:49<00:00, 45.77it/s]

Processed 5000 rows in 1663.26 seconds
Epoch 15/15 completed
Average loss: 0.0000
Skipped rows in this epoch: 0
Training completed
Total time elapsed: 1663.27 seconds


4:主函数和训练循环

In [1]:
def retrieve_images(query, model, criterion, k=5):
    query_features = extract_text_features(query)
    query_output, _ = model(query_features, None)

    similarities = []
    for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Retrieving images"):
        path = str(row['path'])
        img_features = extract_image_features(os.path.join(image_folder, path))
        if img_features is None:
            continue
        _, img_output = model(None, img_features)
        similarity = criterion(query_output, img_output)
        similarities.append((path, similarity.item()))

    return sorted(similarities, key=lambda x: x[1], reverse=True)[:k]

# 处理测试查询
results = []
for query in tqdm(test_df['title'], desc="Processing test queries"):
    top_k_images = retrieve_images(query, trained_model, trained_criterion)
    row = [query] + [img for img, _ in top_k_images]
    # 确保每行都有 6 个元素（1个查询 + 5个图像）
    while len(row) < 6:
        row.append('')  # 如果检索到的图像少于5个，用空字符串填充
    results.append(row[:6])  # 只取前6个元素，以防万一

# 打印一些调试信息
print(f"Number of results: {len(results)}")
print(f"Sample result: {results[0]}")

# 创建提交文件
try:
    submission_df = pd.DataFrame(results, columns=['title', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5'])
    submission_df.to_csv('submit.csv', index=False, encoding='utf-8')
    print("Submission file created: submit.csv")
except Exception as e:
    print(f"Error creating submit file: {str(e)}")
    print(f"Results shape: {len(results)} rows, {len(results[0]) if results else 0} columns")

# 下载提交文件
files.download('submit.csv')

NameError: name 'tqdm' is not defined

5:图像检索和结果生成